In [10]:
import random
from openai import OpenAI
import networkx as nx
import time
import pickle
import os
client = OpenAI(api_key='KEY')


In [11]:
def create_ring_lattice(n, k):
    G = nx.Graph()
    G.add_nodes_from(range(n))

    for j in range(1, k // 2 + 1):
        targets = [i + j for i in range(n)]
        for i in range(n):
            G.add_edge(i, targets[i] % n)
    return G

def shuffle(G):
    nodes = list(G.nodes())
    shuffled_nodes = nodes.copy()
    random.shuffle(shuffled_nodes)

    mapping = {nodes[i]: shuffled_nodes[i] for i in range(len(nodes))}
    G_shuffled = nx.relabel_nodes(G, mapping)

    return G_shuffled

In [13]:
initial_dialogue_history = [{"role": "system", "content": '''Pretend you are a human playing a prisoner's dilemma game in lab environment. I will now explain the rules to you, are you ready?'''}]

def add_message(role, content, dialogue_history):
    """Add a new message to the dialogue history."""
    temp = dialogue_history.copy()
    temp.append({"role": role, "content": content})
    return temp

def get_openai_response(dialogue_history, gpt_model, max_token):
    """Get a response from OpenAI using the current dialogue history with retries on timeout."""
    timeout = 30  # Set your desired timeout in seconds
    start_time = time.time()
    
    while True:
        try:
            response =  client.chat.completions.create(
                model=gpt_model,  # gpt-3.5-turbo, gpt-4-1106-preview
                messages=dialogue_history,
                max_tokens=max_token
            )
            return response.choices[0].message.content

        except Exception as e:
            print(e)
            if time.time() - start_time > timeout:
                raise TimeoutError("Request to OpenAI timed out.") from e
            print("Request failed, retrying...")
            time.sleep(1) 

def inquire_choice(player, round, choices):
    #print('inquiry player: ',player)
    return choices[player][round-1]

def make_summary(player ,choices, round, G, benefit, cost):
    paid = 0
    choice = inquire_choice(player, round, choices)
    first_line = ""
    
    gain = 0
    if choice == 'C':
        paid = cost * len(G[player])
    for neighbor in G[player]:
        
        if inquire_choice(neighbor, round, choices) == 'C':
            gain += benefit
    
    total_payoff = gain - paid
    first_line = """Last round you chose %s, paid %s points, and gained %s points, for a total of %s points.""" %(choice, paid, gain, total_payoff)
    
    prompt = ""
    i = 0
    for neighbor in G[player]:
        #print(player, neighbor)
        gain_from_the_player = 0
        gain_from_others = 0
        this_neighbor_paid = 0
        this_neighbors_choice = inquire_choice(neighbor, round, choices)
        # print(round,player, neighbor, this_neighbors_choice)
        if this_neighbors_choice == 'C':
            this_neighbor_paid = cost * len(G[neighbor])
        if choice == 'C':
            gain_from_the_player += benefit
        for neighbours_neighbour in G[neighbor]:
            if neighbours_neighbour != player:
                if inquire_choice(neighbours_neighbour, round, choices) == 'C':
                    gain_from_others += benefit
        i += 1
        this_neighbors_total_payoff =  gain_from_the_player + gain_from_others - this_neighbor_paid
        #print(this_neighbors_total_payoff)
        prompt += """Last round, neighbor %s chose %s, paid %s points, and gained %s points from you, and %s points from other players, for a total of %s points."""%(neighbor,this_neighbors_choice, this_neighbor_paid, gain_from_the_player, gain_from_others, this_neighbors_total_payoff)
    
    return (first_line+prompt), total_payoff

def make_prompt(choices, player, round, G, benefit, cost):
    last_line = """Please choose for the current round. """
    summary, payoff = make_summary(player,choices, round, G, benefit, cost) 
    summary += last_line
    return summary, payoff

In [14]:
def make_initial_prompts(n_rounds, cost, benefit, network_structured):

    initial_prompts = []
    if network_structured:
        initial_prompts.append("""The game will be played over a series of rounds. In every round you make a choice about whether to pay to give points to the other players you are connected to. The game will last for %s rounds. You will be connected to the same people every round. We now describe the game in more details. Are you ready?"""%(n_rounds)) 
    else: 
        initial_prompts.append("""The game will be played over a series of rounds. In every round you make a choice about whether to pay to give points to the other players you are connected to. The game will last for %s rounds. Your neighbors will be randomized between every round. We now describe the game in more details. Are you ready?"""%(n_rounds))

    initial_prompts.append("""In every round you choose whether to pay to give points to the people you are connected to. If you choose to cooperate (represented by 'C'), you pay %s points for each player you are connected to and each of them gain %s points. If you choose to defect (represented by 'D'), you do not pay any points and do not change the points of the players you are connected to. Each player you are connected to has the same choice. For each of them that chooses 'C', you gain %s points. Once everyone makes a decision, I will tell you the result of the round. You will be shown the choices of each player you are connected to and how many points in total you gained or lost. You will also be shown how many points each player you are connected to gained or lost in total. These numbers are affected by your choice, their choice, and also the choices of any other players connected by them who may or may not be connected to you. Remember, for every 300 points you have at the end of the game, we will pay you 1 dollar. You should aim for getting as much points as you can. Do you understand?""" % (cost, benefit,benefit))
    initial_prompts.append("""You have now complete the tutorial. Are you ready?""")
    initial_prompts.append("""Thank you for completing the tutorial. You will now be playing with other participants. Please only give your choice with 'D' or 'C'. Are you ready?""")
    initial_prompts.append("""Please make choice for the first round. Please only reply your choice with 'D' or 'C'. """)
    return initial_prompts

In [15]:
def play(N, n_neighbour, fixed, n_rounds, gpt_model, benefit, cost, initial_prompts):
    G = create_ring_lattice(N,n_neighbour)
    players = list(G.nodes)
    prompts = {}
    dialogue_histories = {}
    choices = {player: [] for player in players}
    payoff_frq_C = []
    payoff_frq_D = []

    for player in players:
        prompts[player] = initial_prompts
        dialogue_histories[player] = initial_dialogue_history
        #print(dialogue_histories)
    # print(players)
    
    for i in range(n_rounds+len(initial_prompts)):
        print(i)
        if i < len(initial_prompts):
            max_tokens = 20
            for player in players:
                
                openai_response = get_openai_response(dialogue_histories[player],gpt_model,max_tokens)
                temp = add_message("assistant", openai_response,dialogue_histories[player])
                dialogue_histories[player] = temp
                #print(prompts)

                user_input = prompts[player][i]
                temp = add_message("user", user_input,dialogue_histories[player])
                dialogue_histories[player] = temp
                

        else:
            try:
                max_tokens = 1
                for player in players:
                    openai_response = get_openai_response(dialogue_histories[player],gpt_model,max_tokens)
                    temp = add_message("assistant", openai_response,dialogue_histories[player])
                    dialogue_histories[player] = temp
                    while openai_response != 'C' and openai_response != 'D':
                        user_input = 'Please answer with C or D representing Cooperate or Defect.'
                        temp = add_message("user", user_input,dialogue_histories[player])
                        dialogue_histories[player] = temp
                        openai_response = get_openai_response(dialogue_histories[player],gpt_model,max_tokens)
                        temp = add_message("assistant", openai_response,dialogue_histories[player])
                        dialogue_histories[player] = temp
                    
                    choices_history = choices[player]
                    #print(openai_response,choices_history)
                    choices_history.append(openai_response)
                    choices[player] = choices_history
                    #print(choices_history,choices)

                
                for player in players:
                    prompts_history = prompts[player].copy()
                    # print(choices,player,i-4)
                    
                    
                    new_prompt, payoff = make_prompt(choices,player,i-len(initial_prompts)+1, G, benefit, cost)
                    prompts_history.append(new_prompt)
                    prompts[player] = prompts_history
                    #print(choices, i-len(initial_prompts))
                    if choices[player][i-len(initial_prompts)] == 'C':
                        payoff_frq_C.append(payoff)
                    elif choices[player][i-len(initial_prompts)] == 'D':
                        payoff_frq_D.append(payoff)
                    else:
                        print('Error', choices[player][i-len(initial_prompts)])

                    # print(player)
                    # print(prompts[player])
                    # print(new_prompt)
                    # print(payoff)


                #print(prompts)
                for player in players:
         
                    user_input = prompts[player][i]
                    
                    temp = add_message("user", user_input,dialogue_histories[player])
                    dialogue_histories[player] = temp
            except Exception as e:
                print(e)
                return choices, dialogue_histories, G, payoff_frq_C, payoff_frq_D

        if not fixed:
            G = shuffle(G)
    return choices, dialogue_histories, G, payoff_frq_C, payoff_frq_D
    # return dialogue_histories



In [16]:
def write_dict(name, path, dict):
    with open('%s/%s.txt'%(path,name), 'w') as file:
        for key in dict:
            file.write('%s:%s\n' % (key, dict[key]))

def write_list(name, path, my_list):
    with open('%s/%s.txt'%(path,name), 'w') as file:
        # Write each item on a new line
        for item in my_list:
            file.write(f"{item}\n")

def write_G(name, path, G):
    with open("%s/%s.pkl"%(path, name), "wb") as f:
        pickle.dump(G, f)

In [31]:

k_lst = [2]
c = 10
n_rounds = 25
b_lst = [60]

gpt3 = 'gpt-3.5-turbo'
gpt4 = 'gpt-4-1106-preview'
gpt_model = gpt3 # gpt-3.5-turbo, gpt-4-1106-preview, text-davinci-003，gpt-3.5-turbo-1106
network_structured = False
n_players = 15
root_path = 'PATH'


for k in k_lst:
    for b in b_lst:
        initial_prompts =  make_initial_prompts(n_rounds, c, b, network_structured)
        b_c = b/c
        model = ''
        if gpt_model == gpt3:
            model = 'gpt3.5'
        elif gpt_model == gpt4:
            model = 'gpt4'
        else:
            model = gpt_model

        if b_c > k:

            choices, dialogue_histories, G, payoff_frq_C, payoff_frq_D = play(n_players,k,network_structured, n_rounds, gpt_model, b, c, initial_prompts)
            structure = 'n' if network_structured == True else 'm'
            path = root_path+'/results/%s/r=%s/#p=%s/%s_%s_%s'%(model,n_rounds,n_players,b_c, k, structure)
            if not os.path.exists(path):
                os.makedirs(path)
            write_dict('choices', path, choices)
            write_dict('dialogs', path, dialogue_histories)
            write_G('G', path, G)
            write_list('payoff_frq_C', path, payoff_frq_C)
            write_list('payoff_frq_D', path, payoff_frq_D)
        print('-------------------------------------------')
    print("===============================================================================")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
-------------------------------------------
